# xLSTM Music Generation - Clean Pipeline

This notebook provides a clean, modular approach to generating music with your trained xLSTM model.

## Key Fixes from Your Original Code:

1. **Memory Issue Fixed**: The problem was `max_length` growing with each iteration
   - **Wrong**: `max_length = len(output.split()) + chunk_size` (creates quadratic memory growth)
   - **Right**: Use fixed `max_length` OR sliding window context

2. **Context Length**: You can use larger context during inference than training
   - Trained with 2048 → Can infer with 4096 or more
   - But memory grows with context² in mLSTM

3. **Modular Design**: Clean separation of generation and conversion logic

In [1]:
print("Hello0")

Hello0


## Setup

In [2]:
import sys
sys.path.append("/scratch1/e20-fyp-xlstm-music-generation/e20fyptemp1/fyp-musicgen/repos/helibrunna")

from xlstm_music_generation import MusicGenerator, MIDIConverter, generate_music
from pathlib import Path

import os
os.environ['TORCH_CUDA_ARCH_LIST'] = '8.0;8.6;8.9'
os.environ['MAX_JOBS'] = '4'

/home/e20037/miniconda/envs/xlstm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Initialize Generator

In [3]:
# MODEL_PATH = "/scratch1/e20-fyp-xlstm-music-generation/e20fyptemp1/fyp-musicgen/repos/helibrunna/output/lmd_remigen_xlstm/run_20260115-1028"

# Context-2048, embedding-512
MODEL_PATH = "/scratch1/e20-fyp-xlstm-music-generation/e20fyptemp1/fyp-musicgen/repos/helibrunna/output/xlstm_lmd_512d_2048ctx_12b/run_20260126-0516"

# For short sequences (< 2048 tokens)
generator = MusicGenerator(
    model_path=MODEL_PATH,
    context_length=16_384,
    device="cuda"
)

converter = MIDIConverter()

Loading model from: /scratch1/e20-fyp-xlstm-music-generation/e20fyptemp1/fyp-musicgen/repos/helibrunna/output/xlstm_lmd_512d_2048ctx_12b/run_20260126-0516
   ▄█    █▄       ▄████████  ▄█        ▄█  ▀█████████▄     ▄████████ ███    █▄  ███▄▄▄▄   ███▄▄▄▄      ▄████████ 
  ███    ███     ███    ███ ███       ███    ███    ███   ███    ███ ███    ███ ███▀▀▀██▄ ███▀▀▀██▄   ███    ███ 
  ███    ███     ███    █▀  ███       ███▌   ███    ███   ███    ███ ███    ███ ███   ███ ███   ███   ███    ███ 
 ▄███▄▄▄▄███▄▄  ▄███▄▄▄     ███       ███▌  ▄███▄▄▄██▀   ▄███▄▄▄▄██▀ ███    ███ ███   ███ ███   ███   ███    ███ 
▀▀███▀▀▀▀███▀  ▀▀███▀▀▀     ███       ███▌ ▀▀███▀▀▀██▄  ▀▀███▀▀▀▀▀   ███    ███ ███   ███ ███   ███ ▀███████████ 
  ███    ███     ███    █▄  ███       ███    ███    ██▄ ▀███████████ ███    ███ ███   ███ ███   ███   ███    ███ 
  ███    ███     ███    ███ ███▌    ▄ ███    ███    ███   ███    ███ ███    ███ ███   ███ ███   ███   ███    ███ 
  ███    █▀      ██████████ █████▄▄██ █▀   ▄███

Using /home/e20037/.cache/torch_extensions/py311_cu124 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/e20037/.cache/torch_extensions/py311_cu124/slstm_HS512BS8NH4NS4DBfDRbDWbDGbDSbDAfNG4SA1GRCV0GRC0d0FCV0FC0d0/build.ninja...
Building extension module slstm_HS512BS8NH4NS4DBfDRbDWbDGbDSbDAfNG4SA1GRCV0GRC0d0FCV0FC0d0...
Using envvar MAX_JOBS (4) as the number of workers...


ninja: no work to do.


Loading extension module slstm_HS512BS8NH4NS4DBfDRbDWbDGbDSbDAfNG4SA1GRCV0GRC0d0FCV0FC0d0...
/home/e20037/miniconda/envs/xlstm/lib/python3.11/site-packages/xlstm/blocks/slstm/cell.py:543: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @conditional_decorator(
/home/e20037/miniconda/envs/xlstm/lib/python3.11/site-packages/xlstm/blocks/slstm/cell.py:568: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @conditional_decorator(


{'verbose': True, 'with_cuda': True, 'extra_ldflags': ['-L/home/e20037/miniconda/envs/xlstm/lib', '-lcublas'], 'extra_cflags': ['-DSLSTM_HIDDEN_SIZE=512', '-DSLSTM_BATCH_SIZE=8', '-DSLSTM_NUM_HEADS=4', '-DSLSTM_NUM_STATES=4', '-DSLSTM_DTYPE_B=float', '-DSLSTM_DTYPE_R=__nv_bfloat16', '-DSLSTM_DTYPE_W=__nv_bfloat16', '-DSLSTM_DTYPE_G=__nv_bfloat16', '-DSLSTM_DTYPE_S=__nv_bfloat16', '-DSLSTM_DTYPE_A=float', '-DSLSTM_NUM_GATES=4', '-DSLSTM_SIMPLE_AGG=true', '-DSLSTM_GRADIENT_RECURRENT_CLIPVAL_VALID=false', '-DSLSTM_GRADIENT_RECURRENT_CLIPVAL=0.0', '-DSLSTM_FORWARD_CLIPVAL_VALID=false', '-DSLSTM_FORWARD_CLIPVAL=0.0', '-U__CUDA_NO_HALF_OPERATORS__', '-U__CUDA_NO_HALF_CONVERSIONS__', '-U__CUDA_NO_BFLOAT16_OPERATORS__', '-U__CUDA_NO_BFLOAT16_CONVERSIONS__', '-U__CUDA_NO_BFLOAT162_OPERATORS__', '-U__CUDA_NO_BFLOAT162_CONVERSIONS__'], 'extra_cuda_cflags': ['-Xptxas="-v"', '-gencode', 'arch=compute_80,code=compute_80', '-res-usage', '--use_fast_math', '-O3', '-Xptxas -O3', '--extra-device-vectori

Using /home/e20037/.cache/torch_extensions/py311_cu124 as PyTorch extensions root...
No modifications detected for re-loaded extension module slstm_HS512BS8NH4NS4DBfDRbDWbDGbDSbDAfNG4SA1GRCV0GRC0d0FCV0FC0d0, skipping build step...
Loading extension module slstm_HS512BS8NH4NS4DBfDRbDWbDGbDSbDAfNG4SA1GRCV0GRC0d0FCV0FC0d0...


{'verbose': True, 'with_cuda': True, 'extra_ldflags': ['-L/home/e20037/miniconda/envs/xlstm/lib', '-lcublas'], 'extra_cflags': ['-DSLSTM_HIDDEN_SIZE=512', '-DSLSTM_BATCH_SIZE=8', '-DSLSTM_NUM_HEADS=4', '-DSLSTM_NUM_STATES=4', '-DSLSTM_DTYPE_B=float', '-DSLSTM_DTYPE_R=__nv_bfloat16', '-DSLSTM_DTYPE_W=__nv_bfloat16', '-DSLSTM_DTYPE_G=__nv_bfloat16', '-DSLSTM_DTYPE_S=__nv_bfloat16', '-DSLSTM_DTYPE_A=float', '-DSLSTM_NUM_GATES=4', '-DSLSTM_SIMPLE_AGG=true', '-DSLSTM_GRADIENT_RECURRENT_CLIPVAL_VALID=false', '-DSLSTM_GRADIENT_RECURRENT_CLIPVAL=0.0', '-DSLSTM_FORWARD_CLIPVAL_VALID=false', '-DSLSTM_FORWARD_CLIPVAL=0.0', '-U__CUDA_NO_HALF_OPERATORS__', '-U__CUDA_NO_HALF_CONVERSIONS__', '-U__CUDA_NO_BFLOAT16_OPERATORS__', '-U__CUDA_NO_BFLOAT16_CONVERSIONS__', '-U__CUDA_NO_BFLOAT162_OPERATORS__', '-U__CUDA_NO_BFLOAT162_CONVERSIONS__'], 'extra_cuda_cflags': ['-Xptxas="-v"', '-gencode', 'arch=compute_80,code=compute_80', '-res-usage', '--use_fast_math', '-O3', '-Xptxas -O3', '--extra-device-vectori

Using /home/e20037/.cache/torch_extensions/py311_cu124 as PyTorch extensions root...
No modifications detected for re-loaded extension module slstm_HS512BS8NH4NS4DBfDRbDWbDGbDSbDAfNG4SA1GRCV0GRC0d0FCV0FC0d0, skipping build step...
Loading extension module slstm_HS512BS8NH4NS4DBfDRbDWbDGbDSbDAfNG4SA1GRCV0GRC0d0FCV0FC0d0...
/home/e20037/miniconda/envs/xlstm/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2165: FutureWarning: Calling PreTrainedTokenizerFast.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


✓ Model loaded (context: 16384 tokens)


/home/e20037/miniconda/envs/xlstm/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Batch Generation

In [13]:
batch_results = generator.generate_batch(
    temperatures=[0.6, 0.7, 0.8, 0.9],
    target_bars_list=[16, 32, 48, 64],
    pieces_per_combination=5,
    verbose=True
)

BATCH GENERATION
Temperatures: [0.6, 0.7, 0.8, 0.9]
Target bars: [16, 32, 48, 64]
Pieces per combination: 5
Total pieces: 80
Output directory: ./generated_batch_20260208_041715

[1/80] Generating temp_0.6_bars_16_001.mid...
   ✓ Success - 30 bars, 780 tokens
   Time: 20.8s, Cleaning: No

[2/80] Generating temp_0.6_bars_16_002.mid...
   ✓ Success - 17 bars, 1690 tokens
   Time: 71.8s, Cleaning: No

[3/80] Generating temp_0.6_bars_16_003.mid...
   ✓ Success - 20 bars, 1480 tokens
   Time: 52.0s, Cleaning: No

[4/80] Generating temp_0.6_bars_16_004.mid...
   ✓ Success - 19 bars, 1138 tokens
   Time: 34.1s, Cleaning: No

[5/80] Generating temp_0.6_bars_16_005.mid...
   ✓ Success - 18 bars, 1448 tokens
   Time: 51.2s, Cleaning: No

[6/80] Generating temp_0.6_bars_32_001.mid...
✗ Decoding error: AssertionError: 
   First attempt failed, trying with cleaning...
   ✓ Success - 32 bars, 4340 tokens
   Time: 259.5s, Cleaning: Yes

[7/80] Generating temp_0.6_bars_32_002.mid...
✗ Decoding error: A

In [5]:
prompt = "s-9 o-42 t-36"

In [9]:
result = generator.generate_long(
    prompt=prompt,
    temperature=0.7,
    target_bars=32,
    verbose=True
)

🎵 Long generation (bar-aware chunking)...
   Target: 32 bars
   Strategy: Generate 2-3 bars per iteration, cut at b-1

📝 Iteration 1/50
   Context: 3 tokens (0 bars)
   Generated: 400 tokens
   Kept (complete bars): 307 tokens (4 bars)
   Total: 310 tokens (4 bars)

📝 Iteration 2/50
   Context: 310 tokens (4 bars)
   Generated: 400 tokens
   Kept (complete bars): 400 tokens (4 bars)
   Total: 710 tokens (8 bars)

📝 Iteration 3/50
   Context: 710 tokens (8 bars)
   Generated: 400 tokens
   Kept (complete bars): 400 tokens (4 bars)
   Total: 1110 tokens (12 bars)

📝 Iteration 4/50
   Context: 1110 tokens (12 bars)
   Generated: 400 tokens
   Kept (complete bars): 400 tokens (4 bars)
   Total: 1510 tokens (16 bars)

📝 Iteration 5/50
   Context: 1500 tokens (16 bars)
   Generated: 398 tokens
   Kept (complete bars): 398 tokens (4 bars)
   Total: 1908 tokens (20 bars)

📝 Iteration 6/50
   Context: 1500 tokens (16 bars)
   Generated: 399 tokens
   Kept (complete bars): 399 tokens (4 bars)
  

In [10]:
# Analyze
analysis = generator.analyze_tokens(result['tokens'], verbose=True)

# Check for issues
if analysis['has_errors']:
    print(f"Found {len(analysis['grammar_errors'])} grammar errors!")

TOKEN ANALYSIS REPORT

📊 OVERALL STATISTICS:
   Total tokens: 3292
   Total bars: 33
   Total notes: 698
   Unique instruments: 3

🎵 BAR STATISTICS:
   Average bar length: 98.8 tokens
   Min bar length: 13 tokens
   Max bar length: 169 tokens

🔤 TOKEN TYPES:
   b-: 33
   d-: 698
   i-: 553
   o-: 290
   p-: 698
   s-: 33
   t-: 290
   v-: 697

🎹 INSTRUMENTS:
   i-128
   i-29
   i-30

✅ SEQUENCE HEALTH:
   Ends with b-1: ✓
   Grammar errors: 4

⚠️  GRAMMAR ERRORS (showing first 5):
   - Incomplete triplet at token 1696: p-53 d-4 (missing velocity)
   - Incomplete triplet at token 1899: p-166 d-2 (missing velocity)
   - Orphan token at 2248: d-4
   - Incomplete triplet at token 2628: p-164 (missing duration)

🔍 SEQUENCE EDGES:
   First 10 tokens: s-9 o-42 t-36 i-128 p-166 d-2 v-28 o-45 t-36 i-128
   Last 10 tokens: d-2 v-31 p-64 d-2 v-31 i-128 p-166 d-2 v-28 b-1
Found 4 grammar errors!


In [11]:
# Try decoding WITHOUT cleaning
success = converter.tokens_to_midi(result['tokens'], "./output/long_clean.mid", clean=False)
print(f"Success: {success}")

✗ Decoding error: AssertionError: 
Success: False


In [12]:
# Try decoding WITH cleaning
success = converter.tokens_to_midi(result['tokens'], "./output/long_clean.mid", clean=True)
print(f"Success: {success}")

Success: True


In [ ]:
# Debug the last tokens
tokens_list = result['tokens'].split()
print(f"Total: {len(tokens_list)}")
print(f"Last 30 tokens: {tokens_list[-30:]}")
print(f"Ends with b-1? {tokens_list[-1] == 'b-1'}")


In [4]:
# Try single-shot generation with larger max_length
result = generator.generate(
    prompt="s-9 o-0 t-35",
    temperature=0.8,
    max_tokens=2000,
    verbose=True
)


🎵 Generating...
   Prompt: s-9 o-0 t-35...
   Max tokens: 2000
   Temperature: 0.8
✓ Generated 2000 tokens (10 bars)


In [ ]:
# Save
output_path = "./output/ctx2048-full-trained/sample_1_ct4096-len1990.mid"
success = converter.tokens_to_midi(result['tokens'], output_path, clean=False)

if success:
    print(f"✓ Saved: {output_path}")
    print(f"Generated {result['num_tokens']} tokens, {result['bars']} bars")
else:
    print("Decoding failed")

✓ Saved: ./output/ctx4096-full-trained/sample_1_ct4096-len1990.mid
Generated 2000 tokens, 10 bars


In [ ]:
# Debug: see what token is causing the error
tokens = result['tokens'].split()
print(f"Total tokens: {len(tokens)}")
print(f"Last 30 tokens: {tokens[-30:]}")

# Find incomplete triplets
for i, token in enumerate(tokens[-30:], start=len(tokens)-30):
    if token.startswith('p-'):
        if i+1 >= len(tokens) or not tokens[i+1].startswith('d-'):
            print(f"Incomplete at {i}: {token} (no duration)")
        elif i+2 >= len(tokens) or not tokens[i+2].startswith('v-'):
            print(f"Incomplete at {i}: {token} {tokens[i+1]} (no velocity)")

## Example 1: Generate Short Piece

In [ ]:
# Simple generation
result = generator.generate(
    prompt="s-9 o-0 t-38",
    temperature=0.8,
    max_tokens=3000,
    verbose=True
)

print(f"\nGenerated {result['num_tokens']} tokens, {result['bars']} bars")

# Convert to MIDI
output_path = "./output/test_song.mid"
converter.tokens_to_midi(result['tokens'], output_path)
print(f"Saved to: {output_path}")

## Example 2: Generate Long Piece (Chunked)

This uses **sliding window** approach to avoid memory issues.

In [ ]:
# For long generation, use larger context
long_generator = MusicGenerator(
    model_path=MODEL_PATH,
    context_length=4096,  # Larger than training
    device="cuda"
)

result = long_generator.generate_long(
    prompt="s-9 o-30 t-33 i-128 p-176 d-6 v-23 o-36 t-33 i-128 p-173 d-6 v-23 o-42 t-33 i-128 p-171 d-6 v-23 b-1 s-9 o-0 t-33 i-4 p-81 d-25",
    temperature=0.8,
    target_bars=64,       # Generate 64 bars
    chunk_tokens=1024,    # 1024 new tokens per iteration
    max_iterations=2,
    verbose=True
)

# Save
output_path = "./output/long_song.mid"
converter.tokens_to_midi(result['tokens'], output_path)
print(f"\nSaved {result['bars']} bars to: {output_path}")

In [ ]:
# Add detailed error reporting
cleaned = converter.clean_tokens(result['tokens'])
print(f"Cleaned: {len(cleaned.split())} tokens")
print(f"Last 20 cleaned: {cleaned.split()[-20:]}")

try:
    midi_obj = converter.decoder.decode_from_token_str_list(cleaned.split())
    print("✓ Decoding successful!")
except Exception as e:
    print(f"Error: {type(e).__name__}: {e}")
    import traceback
    traceback.print_exc()

## Example 3: Batch Generation with Different Temperatures

In [ ]:
temperatures = [0.5, 0.8, 1.0, 1.2]
output_dir = Path("./output/temp_comparison")
output_dir.mkdir(exist_ok=True, parents=True)

for temp in temperatures:
    print(f"\n{'='*60}")
    print(f"Temperature: {temp}")
    print('='*60)
    
    result = generator.generate(
        prompt="s-9 o-0 t-35",
        temperature=temp,
        max_tokens=2000,
        verbose=True
    )
    
    midi_path = output_dir / f"temp_{temp:.1f}.mid"
    converter.tokens_to_midi(result['tokens'], str(midi_path))
    print(f"✓ Saved: {midi_path}")

## Example 4: Simple API (One Function)

In [ ]:
# Generate 5 songs with one function call
outputs = generate_music(
    model_path=MODEL_PATH,
    num_songs=5,
    max_tokens=2048,
    temperature=0.8,
    output_dir="./output/batch"
)

print(f"\nGenerated {len(outputs)} songs")

## Example 5: Long Mode with Simple API

In [ ]:
# Generate long pieces
outputs = generate_music(
    model_path=MODEL_PATH,
    num_songs=2,
    temperature=0.8,
    output_dir="./output/long_batch",
    long_mode=True,
    target_bars=64
)

print(f"\nGenerated {len(outputs)} long songs")

## Analyzing Generated MIDI

In [ ]:
import pretty_midi

def analyze_midi(midi_path):
    midi = pretty_midi.PrettyMIDI(str(midi_path))
    
    print(f"File: {midi_path.name}")
    print(f"Duration: {midi.get_end_time():.2f}s")
    print(f"Instruments: {len(midi.instruments)}")
    print(f"Total notes: {sum(len(inst.notes) for inst in midi.instruments)}")
    
    for inst in midi.instruments:
        inst_type = "Drums" if inst.is_drum else f"Program {inst.program}"
        print(f"  - {inst.name}: {len(inst.notes)} notes ({inst_type})")
    print()

# Analyze all generated files
for midi_file in Path("./output").glob("**/*.mid"):
    analyze_midi(midi_file)

## Understanding Context Length

### Training vs Inference:
- **Training**: Model was trained with `context_length=2048`
- **Inference**: You can use `context_length=4096` or higher
  - The model can handle longer sequences
  - But memory usage grows quadratically (N² for mLSTM)

### Memory Usage:
- `context_length=2048` → ~10GB VRAM
- `context_length=4096` → ~40GB VRAM  
- `context_length=8192` → ~160GB VRAM (likely OOM)

### Solution for Long Generation:
Use **sliding window** (implemented in `generate_long()`):
- Keep only last N tokens as context
- Generate new chunk
- Slide window forward
- Repeat

This keeps memory constant while generating arbitrarily long sequences.

## Custom Prompts

REMIGEN format: `s-X o-Y t-Z i-A p-B d-C v-D ...`

- `s-X`: Signature (time signature)
- `o-Y`: Offset (timing)
- `t-Z`: Tempo
- `i-A`: Instrument
- `p-B`: Pitch
- `d-C`: Duration
- `v-D`: Velocity
- `b-1`: Bar marker

In [ ]:
# Try different starting prompts
prompts = [
    "s-9 o-0 t-35",  # Slow tempo
    "s-9 o-0 t-120", # Fast tempo
    "s-9 o-0 t-60 i-0",  # With piano
]

for i, prompt in enumerate(prompts):
    result = generator.generate(
        prompt=prompt,
        temperature=0.8,
        max_tokens=1500,
        verbose=True
    )
    
    midi_path = f"./output/custom_prompt_{i}.mid"
    converter.tokens_to_midi(result['tokens'], midi_path)
    print(f"Saved: {midi_path}\n")

## Comparison with Museformer

For your research comparison, you can now:

1. Generate same number of pieces from both models
2. Use same prompts/seeds
3. Compare:
   - Musicality
   - Coherence over long sequences
   - Diversity
   - Computational requirements

In [ ]:
# Generate evaluation dataset
eval_output = Path("./evaluation/xlstm_samples")
eval_output.mkdir(exist_ok=True, parents=True)

for i in range(20):  # Generate 20 samples
    result = generator.generate(
        prompt="s-9 o-0 t-35",
        temperature=0.8,
        max_tokens=2048,
        verbose=False
    )
    
    midi_path = eval_output / f"xlstm_{i:03d}.mid"
    converter.tokens_to_midi(result['tokens'], str(midi_path))
    
    if (i + 1) % 5 == 0:
        print(f"Generated {i+1}/20 samples")

print(f"\n✓ Evaluation dataset ready: {eval_output}")